# Length 7 and 10 CAZAC Experiments
This notebook runs the first two tests described in the paper where we search for all CAZAC sequences of length 7 and of length 10.

# Imports

In [ ]:
import numpy as np
from tqdm import tqdm
import scipy.optimize as opt
import matplotlib.pyplot as plt

# Function Definitions
We seperated the ambiguity function and autocorrelation function definitions for convenience. If one is only interested in the autocorrelations this saves significant time by not computing the nonzero Doppler shift values of the ambiguity function.

In [ ]:
def cazac_system(x):
    n = int(len(x)/2)
    a = x[:n]
    b = x[n:]
    F = np.zeros(3*n-2)
    F[:n] = a**2 + b**2 - 1
    for k in np.arange(1,n):
        F[n+k-1] = np.sum(a*np.roll(a,k)+b*np.roll(b,k))
        F[2*n-2+k] = np.sum(a*np.roll(b,k)-b*np.roll(a,k))
    return F

def aperiodic_ambiguity(seq):
    n = len(seq)
    AF = np.zeros((2*n-1,n),dtype='complex')
    M = np.eye(n)*np.exp(2*np.pi*1j*np.arange(n)/n)
    
    # Negative time shift side
    for k in np.arange(1,n):
        for l in np.arange(n):
            translate_seq = np.zeros(n,dtype='complex')
            for j in np.arange(k,n):
                translate_seq[j] = seq[j-k]
            shift_seq = np.conjugate(np.linalg.matrix_power(M,l)@seq)
            AF[n-1-k,l] = np.dot(translate_seq,shift_seq)
    # Positive time shift side
    for k in np.arange(n):
        for l in np.arange(n):
            translate_seq = np.zeros(n,dtype='complex')
            for j in np.arange(n-k):
                translate_seq[j] = seq[j+k]
            shift_seq = np.conjugate(np.linalg.matrix_power(M,l)@seq)
            AF[n-1+k,l] = np.dot(translate_seq,shift_seq)
    return AF

def aperiodic_autocorrelation(seq):
    n = len(seq)
    AC = np.zeros(2*n-1,dtype='complex')
    
    # Negative time shift side
    for k in np.arange(1,n):
        translate_seq = np.zeros(n,dtype='complex')
        for j in np.arange(k,n):
            translate_seq[j] = seq[j-k]
        conj_seq = np.conjugate(seq)
        AC[n-1-k] = np.dot(conj_seq,translate_seq)
    # Positive time shift side
    for k in np.arange(n):
        translate_seq = np.zeros(n,dtype='complex')
        for j in np.arange(n-k):
            translate_seq[j] = seq[j+k]
        conj_seq = np.conjugate(seq)
        AC[n-1+k] = np.dot(conj_seq,translate_seq)
    return AC

# Length 7 CAZAC Sequence Search
We used a large number of trials to try to enumrate all length 7 CAZACs. For convenience, we summarize what is known about the numbef of various length 7 CAZACs below.

Number of length 7 CAZACs: 532<br>
Roots of unity CAZACs known: 42<br>
Non-roots of unity CAZACs known: Up to 252 (Bjorck)<br>

In [ ]:
trials7 = 10000
rounding7 = 8
cost_factor7 = -10

In [ ]:
opt_result = np.zeros((trials7,7),dtype='complex128')
opt_cost = np.zeros((trials7))
for trial in tqdm(np.arange(trials7)):
    x0 = np.random.rand((14))
    result = opt.least_squares(cazac_system,x0,jac='3-point',ftol=10**(-12),xtol=10**(-12),gtol=10**(-12))
    sequence = result.x[:7]+1j*result.x[7:]
    opt_result[trial,:] = sequence/sequence[0]
    opt_cost[trial] = result.cost

In [ ]:
cost_result = np.reshape(opt_result[np.where(opt_cost < 10**(cost_factor7)),:],(-1,7))
result_round = np.round(cost_result,decimals=rounding7)
result_unique = np.unique(result_round,axis=0)
print('Maximum cost: %.6e'%np.max(opt_cost))
print('Sequences found: %d'%result_unique.shape[0])

# Filtering Out Known Length 7 CAZACs
We wanted to examine the autocorrelation properties of a new length 7 CAZAC sequences to the two major ones already known: Zadoff-Chu and Bjorck. To do this we looked at the orbit of each of those two sequences and removed any
matching CAZAC sequence from the list of 532 sequences found.

In [ ]:
cazac7 = np.genfromtxt('cazac_len7_result.csv',dtype=None)
T = np.zeros((7,7))
M = np.eye(7)*np.exp(2*np.pi*1j*np.arange(7)/7)
for j in np.arange(1,7):
    T[j,j-1]=1
T[0,6] = 1
D = np.zeros((6,7,7))
for m in np.arange(6):
    for j in np.arange(7):
        D[m,j,(m+1)*j%7] = 1

In [ ]:
b = np.exp(1j*np.arccos(-3/4))
B = np.array([1,1,1,b,1,b,b])

In [ ]:
Fcazac7 = np.zeros((532,7),dtype='complex')
Fcazac7[:,:] = cazac7[:,:]
for k in np.arange(7):
    for l in np.arange(7):
        for m in np.arange(6):
            matchMtx = np.linalg.matrix_power(M,l)@np.linalg.matrix_power(T,k)@D[m,:,:]@B
            matchMtx = matchMtx/matchMtx[0]
            rowmatch = np.where(np.sum(np.abs(cazac7 - matchMtx),axis=1) < 10**-5)
            rowmatch_conjugate = np.where(np.sum(np.abs(cazac7 - np.conjugate(matchMtx)),axis=1) < 10**-5)
            Fcazac7[rowmatch,:] = -10
            Fcazac7[rowmatch_conjugate,:] = -10
Fcazac7 = Fcazac7[np.where(Fcazac7[:,0] != -10)[0],:]
print(Fcazac7.shape)

In [ ]:
w = np.exp(2*np.pi*1j/7)
qp = np.array([1,w,w**3,w**6,w**10,w**15,w**21])

In [ ]:
NewCazac7 = np.zeros((Fcazac7.shape[0],7), dtype ='complex')
NewCazac7[:,:] = Fcazac7[:,:]
for k in np.arange(7):
    for l in np.arange(7):
        for m in np.arange(6):
            matchMtx = np.linalg.matrix_power(M,l)@np.linalg.matrix_power(T,k)@D[m,:,:]@qp
            matchMtx = matchMtx/matchMtx[0]
            rowmatch = np.where(np.sum(np.abs(Fcazac7 - matchMtx),axis=1) < 10**-5)
            rowmatch_conjugate = np.where(np.sum(np.abs(Fcazac7 - np.conjugate(matchMtx)),axis=1) < 10**-5)
            NewCazac7[rowmatch,:] = -10
            NewCazac7[rowmatch_conjugate,:] = -10
NewCazac7 = NewCazac7[np.where(NewCazac7[:,0] != -10)[0],:]
np.savetxt('cazac_len7_result_new.csv',NewCazac7,fmt='%.8f')
print(NewCazac7.shape)

# Aperiodic Comparison for Length 7

In [ ]:
# Pick which new CAZAC sequence you want to use. 
#The default here is the one we found with the best autocorrelation properties.
CAZAC_select = 197

chu = np.exp(np.pi*1j*np.arange(7)*(np.arange(7)-1)/7)
AC_chu = aperiodic_autocorrelation(chu)
AC_bjorck = aperiodic_autocorrelation(B)
AC_new = aperiodic_autocorrelation(NewCazac7[CAZAC_select,:])
psl_chu7 = np.max((np.abs(AF_chu[:6])/7,np.abs(AF_chu[7:]/7)))
psl_bjorck7 = np.max((np.abs(AF_bjorck[:6])/7,np.abs(AF_bjorck[7:]/7)))
psl_new7 = np.max((np.abs(AF_new[:6])/7,np.abs(AF_new[7:]/7)))

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(-6+np.arange(13),np.abs(AC_chu),c='#fc8d62',label='Zadoff-Chu')
ax.plot(-6+np.arange(13),np.abs(AC_new),c='#202020',label='New CAZAC')
ax.plot(-6+np.arange(13),np.abs(AC_bjorck),c='#8d90db',label='Bjorck')
ax.set_xlabel(r'Time shift $k$',size='x-large')
ax.set_ylabel(r'$|A_a(x)[k]|$',size='x-large')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(prop={'size': 12})

# Length 10 CAZAC Sequence Search

In [ ]:
trials = 200000
rounding = 8
cost_factor = -10

In [ ]:
opt_result = np.zeros((trials,10),dtype='complex128')
opt_cost = np.zeros((trials))
for trial in tqdm(np.arange(trials)):
    x0 = np.random.rand(20)
    result = opt.least_squares(cazac_system,x0,jac='3-point',ftol=10**(-12),xtol=10**(-12),gtol=10**(-12))
    sequence = result.x[:10]+1j*result.x[10:]
    opt_result[trial,:] = sequence/sequence[0]
    opt_cost[trial] = result.cost

In [ ]:
cost_result = np.reshape(opt_result[np.where(opt_cost < 10**(cost_factor)),:],(-1,10))
result_round = np.round(cost_result,decimals=rounding)
result_unique = np.unique(result_round,axis=0)
print('Maximum cost: %.6e'%np.max(opt_cost))
print('Sequences found: %d'%result_unique.shape[0])